In [12]:
!pip install torch torchvision numpy matplotlib


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from PIL import Image
import os

# 원본 이미지 경로
input_folder = '/content/drive/MyDrive/Colab Notebooks/trainB_1'  # 원본 이미지 폴더 경로
output_folder = '/content/drive/MyDrive/Colab Notebooks/trainB'  # 전처리 후 저장될 폴더 경로

# 전처리 작업 수행
def preprocess_images(input_folder, output_folder, target_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(input_folder, filename)
            with Image.open(image_path) as img:
                # 이미지 리사이즈
                img = img.resize(target_size, Image.BICUBIC)
                # 전처리 후 이미지 저장
                img.save(os.path.join(output_folder, filename))

# 전처리 수행 (한국 동양화 데이터셋)
preprocess_images('/content/drive/MyDrive/Colab Notebooks/trainB_1', '/content/drive/MyDrive/Colab Notebooks/trainB')


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets

# ResNet 기반의 제너레이터
class ResnetGenerator(nn.Module):
    def __init__(self, input_nc, output_nc, n_blocks=9):
        super(ResnetGenerator, self).__init__()
        # 입력 레이어
        model = [nn.Conv2d(input_nc, 64, kernel_size=7, padding=3),
                 nn.InstanceNorm2d(64),
                 nn.ReLU(True)]

        # 다운샘플링 레이어
        for i in range(2):
            mult = 2 ** i
            model += [nn.Conv2d(64 * mult, 64 * mult * 2, kernel_size=3, stride=2, padding=1),
                      nn.InstanceNorm2d(64 * mult * 2),
                      nn.ReLU(True)]

        # ResNet 블록
        mult = 2 ** 2
        for i in range(n_blocks):
            model += [ResnetBlock(64 * mult)]

        # 업샘플링 레이어
        for i in range(2):
            mult = 2 ** (2 - i)
            model += [nn.ConvTranspose2d(64 * mult, int(64 * mult / 2),
                                         kernel_size=3, stride=2,
                                         padding=1, output_padding=1),
                      nn.InstanceNorm2d(int(64 * mult / 2)),
                      nn.ReLU(True)]

        # 출력 레이어
        model += [nn.Conv2d(64, output_nc, kernel_size=7, padding=3)]
        model += [nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

# ResNet Block 정의
class ResnetBlock(nn.Module):
    def __init__(self, dim):
        super(ResnetBlock, self).__init__()
        self.conv_block = self.build_conv_block(dim)

    def build_conv_block(self, dim):
        conv_block = []
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=1),
                       nn.InstanceNorm2d(dim),
                       nn.ReLU(True)]
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=1),
                       nn.InstanceNorm2d(dim)]
        return nn.Sequential(*conv_block)

    def forward(self, x):
        out = x + self.conv_block(x)
        return out


In [14]:
class NLayerDiscriminator(nn.Module):
    def __init__(self, input_nc, ndf=64, n_layers=3):
        super(NLayerDiscriminator, self).__init__()
        kw = 4
        padw = 1
        sequence = [
            nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw),
            nn.LeakyReLU(0.2, True)
        ]

        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):
            nf_mult_prev = nf_mult
            nf_mult = min(2**n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult,
                          kernel_size=kw, stride=2, padding=padw),
                nn.InstanceNorm2d(ndf * nf_mult),
                nn.LeakyReLU(0.2, True)
            ]

        nf_mult_prev = nf_mult
        nf_mult = min(2**n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult,
                      kernel_size=kw, stride=1, padding=padw),
            nn.InstanceNorm2d(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ]

        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)]

        self.model = nn.Sequential(*sequence)

    def forward(self, x):
        return self.model(x)


In [15]:
# 손실 함수 정의
criterion_GAN = nn.MSELoss()  # GAN 손실
criterion_cycle = nn.L1Loss()  # 사이클 일관성 손실
criterion_identity = nn.L1Loss()  # 정체성 손실

# 예시로 사용할 손실 함수 래퍼
def gan_loss(pred, target_is_real):
    target = torch.ones_like(pred) if target_is_real else torch.zeros_like(pred)
    return criterion_GAN(pred, target)

def cycle_consistency_loss(real, rec):
    return criterion_cycle(real, rec)

def identity_loss(real, same):
    return criterion_identity(real, same)


In [16]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

# CustomImageDataset 클래스 정의
class CustomImageDataset(Dataset):  # 'Dataset'을 상속받습니다.
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image

# 이미지 전처리 설정
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(256),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 데이터셋 로드
trainA_dataset = CustomImageDataset('/content/drive/MyDrive/Colab Notebooks/trainB', transform=transform)
trainB_dataset = CustomImageDataset('/content/drive/MyDrive/Colab Notebooks/trainA', transform=transform)

# DataLoader 생성
loader_A = DataLoader(trainA_dataset, batch_size=1, shuffle=True)
loader_B = DataLoader(trainB_dataset, batch_size=1, shuffle=True)


In [18]:
import os
import torchvision.utils as vutils
import torch

# 디바이스 설정 (GPU가 있으면 사용하고, 없으면 CPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 300

import itertools
# 모델 초기화
input_nc = 3  # 입력 채널 수 (RGB 이미지이므로 3)
output_nc = 3  # 출력 채널 수 (목표 이미지도 RGB이므로 3)

G_A2B = ResnetGenerator(input_nc, output_nc).to(device)
G_B2A = ResnetGenerator(input_nc, output_nc).to(device)
D_A = NLayerDiscriminator(input_nc).to(device)
D_B = NLayerDiscriminator(input_nc).to(device)

# 옵티마이저 정의
optimizer_G = optim.Adam(itertools.chain(G_A2B.parameters(), G_B2A.parameters()), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_A = optim.Adam(D_A.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_B = optim.Adam(D_B.parameters(), lr=0.0002, betas=(0.5, 0.999))

# 결과 이미지 저장 폴더 생성 (존재하지 않으면)
os.makedirs('/content/drive/MyDrive/Colab Notebooks/resultA', exist_ok=True)

# 학습 루프
for epoch in range(num_epochs):
    for i, (data_A, data_B) in enumerate(zip(loader_A, loader_B)):
        real_A = data_A[0].to(device)
        real_B = data_B[0].to(device)

        # 제너레이터 업데이트
        optimizer_G.zero_grad()

        fake_B = G_A2B(real_A)
        rec_A = G_B2A(fake_B)

        fake_A = G_B2A(real_B)
        rec_B = G_A2B(fake_A)

        loss_G_A2B = gan_loss(D_B(fake_B), True)
        loss_G_B2A = gan_loss(D_A(fake_A), True)

        loss_cycle_A = cycle_consistency_loss(real_A, rec_A)
        loss_cycle_B = cycle_consistency_loss(real_B, rec_B)

        loss_identity_A = identity_loss(real_A, G_B2A(real_A))
        loss_identity_B = identity_loss(real_B, G_A2B(real_B))

        loss_G = loss_G_A2B + loss_G_B2A + 10 * (loss_cycle_A + loss_cycle_B) + 5 * (loss_identity_A + loss_identity_B)
        loss_G.backward()
        optimizer_G.step()

        # 디스크리미네이터 A 업데이트
        optimizer_D_A.zero_grad()
        loss_D_A = (gan_loss(D_A(real_A), True) + gan_loss(D_A(fake_A.detach()), False)) * 0.5
        loss_D_A.backward()
        optimizer_D_A.step()

        # 디스크리미네이터 B 업데이트
        optimizer_D_B.zero_grad()
        loss_D_B = (gan_loss(D_B(real_B), True) + gan_loss(D_B(fake_B.detach()), False)) * 0.5
        loss_D_B.backward()
        optimizer_D_B.step()

        if i % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}] Batch {i}/{min(len(loader_A), len(loader_B))}")

    # 중간 결과 이미지 저장
    if epoch % 10 == 0:
        with torch.no_grad():
            fake_B = G_A2B(real_A)
            fake_A = G_B2A(real_B)

            # 결과 이미지를 저장 (결과 이미지를 /content/resultA 폴더에 저장)
            vutils.save_image(fake_B, f"/content/drive/MyDrive/Colab Notebooks/resultA/fake_B_epoch_{epoch}.png", normalize=True)
            vutils.save_image(fake_A, f"/content/drive/MyDrive/Colab Notebooks/resultA/fake_A_epoch_{epoch}.png", normalize=True)


Epoch [0/300] Batch 0/39
Epoch [1/300] Batch 0/39
Epoch [2/300] Batch 0/39
Epoch [3/300] Batch 0/39
Epoch [4/300] Batch 0/39
Epoch [5/300] Batch 0/39
Epoch [6/300] Batch 0/39
Epoch [7/300] Batch 0/39
Epoch [8/300] Batch 0/39
Epoch [9/300] Batch 0/39
Epoch [10/300] Batch 0/39
Epoch [11/300] Batch 0/39
Epoch [12/300] Batch 0/39
Epoch [13/300] Batch 0/39
Epoch [14/300] Batch 0/39
Epoch [15/300] Batch 0/39
Epoch [16/300] Batch 0/39
Epoch [17/300] Batch 0/39
Epoch [18/300] Batch 0/39
Epoch [19/300] Batch 0/39
Epoch [20/300] Batch 0/39
Epoch [21/300] Batch 0/39
Epoch [22/300] Batch 0/39
Epoch [23/300] Batch 0/39
Epoch [24/300] Batch 0/39
Epoch [25/300] Batch 0/39
Epoch [26/300] Batch 0/39
Epoch [27/300] Batch 0/39
Epoch [28/300] Batch 0/39
Epoch [29/300] Batch 0/39
Epoch [30/300] Batch 0/39
Epoch [31/300] Batch 0/39
Epoch [32/300] Batch 0/39
Epoch [33/300] Batch 0/39
Epoch [34/300] Batch 0/39
Epoch [35/300] Batch 0/39
Epoch [36/300] Batch 0/39
Epoch [37/300] Batch 0/39
Epoch [38/300] Batch 0

In [19]:
# 학습 후 모델 가중치 저장
torch.save(G_A2B.state_dict(), '/content/drive/MyDrive/Colab Notebooks/resultA/G_A2B.pth')
torch.save(G_B2A.state_dict(), '/content/drive/MyDrive/Colab Notebooks/resultA/G_B2A.pth')

# 모델 초기화
G_A2B = ResnetGenerator(input_nc, output_nc).to(device)
G_B2A = ResnetGenerator(input_nc, output_nc).to(device)

# 저장된 가중치 로드
G_A2B.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/resultA/G_A2B.pth'))
G_B2A.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/resultA/G_B2A.pth'))

# 모델을 평가 모드로 설정
G_A2B.eval()
G_B2A.eval()

ResnetGenerator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (5): ReLU(inplace=True)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (8): ReLU(inplace=True)
    (9): ResnetBlock(
      (conv_block): Sequential(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
        (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): Insta

In [20]:
from PIL import Image
from torchvision import transforms

# 이미지 전처리 설정
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 이미지를 로드하고 전처리
img = Image.open('/content/drive/MyDrive/Colab Notebooks/resultA/56721345-b98b-49e9-b7b1-af1c5f243a74.jpg').convert('RGB')
img_tensor = transform(img).unsqueeze(0).to(device)

# 이미지 변환 (한국 동양화를 서양화로 변환하는 예)
with torch.no_grad():
    output_tensor = G_A2B(img_tensor)

# 이미지 후처리
output_tensor = output_tensor.squeeze().cpu().detach()
output_tensor = (output_tensor + 1) / 2  # [-1, 1] 범위를 [0, 1]로 변환
output_img = transforms.ToPILImage()(output_tensor)

# 이미지 업스케일링 (예: 512x512)
upscale_size = (1920, 1080)  # 원하는 업스케일링 크기
output_img = output_img.resize(upscale_size, Image.BICUBIC)

# 결과 이미지 저장
output_img.save('/content/drive/MyDrive/Colab Notebooks/resultA/output_image_upscaled2.jpg')
